# Optimization for Collabrative filtering

# A. Preprocessing of data

1. Data format

We are going to preprocess a rating file in the following csv format:  
```
UserID::MovieID::Rating::Timestamp
```

2. Prepare data for cross-validation

Splitting Data for a user-based N-fold cross-validation. Store each partition into a new csv file. 

It is convinient to use a python package lenskit. It can be installed by following https://lkpy.readthedocs.io/en/stable/install.html

Use the function lenskit.crossfold.partition_rows to partition all the ratings into N train-test partitions.

3. Convert to a list format

Convert data into a list format for fast processing


In [1]:
import pandas as pd
import numpy as np
import lenskit.crossfold as xf
import csv
from scipy.sparse import bsr_matrix
from scipy.optimize import line_search
from lenskit.metrics.predict import rmse
from lenskit.algorithms.bias import Bias
from lenskit.batch import predict

In [2]:
# read a DataFrame of ratings from the csv ratings file
csvroot = 'data' # TODO
ratings = pd.read_csv(csvroot + '/ratings.csv')
ratings = ratings.rename(columns={'userId': 'user', 'movieId': 'item'})

def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    # use a dictionary to store the index of each element in the list
    unique_dict = {}
    for i in range(len(unique_list)):
        unique_dict[unique_list[i]] = i 
    return unique_list, unique_dict

lst_users, dic_users = unique(ratings['user'])
lst_items, dic_items = unique(ratings['item'])

M = len(lst_users)
N = len(lst_items)
matrixSparsity = len(ratings) / (M*N)
print("We have %d users, %d movies and the rating matrix has %f percent of non-zero value.\n" % (M, N, 100*matrixSparsity))
print(len(ratings))


We have 610 users, 9724 movies and the rating matrix has 1.699968 percent of non-zero value.

100836


In [3]:
# 2 fold cross-validation, store each partition (both train and test) in a seperate csv file.
for i, tp in enumerate(xf.partition_rows(ratings, 5)):
    tp.train.to_csv('train-%d.csv' % (i,))
    tp.test.to_csv('test-%d.csv' % (i,))
    print(tp)


TTPair(train=       user    item  rating
20247   132   46578     4.0
53798   354   48385     3.0
60433   391     308     5.0
17798   111  131013     5.0
87123   562    1259     5.0
...     ...     ...     ...
19702   129    1073     4.0
68132   438    8869     2.0
21196   140    1047     3.0
44420   295    2719     0.5
33010   225    2000     4.0

[80668 rows x 3 columns], test=       user    item  rating
68710   448     377     4.0
40255   274   33679     2.5
7315     50   92046     3.0
49841   318  159976     4.5
63093   414    2340     2.0
...     ...     ...     ...
9084     62  114662     4.0
11303    68   48043     2.0
29960   206    1356     4.0
62666   414    1024     4.0
39022   268    2858     5.0

[20168 rows x 3 columns])
TTPair(train=       user    item  rating
68710   448     377     4.0
40255   274   33679     2.5
7315     50   92046     3.0
49841   318  159976     4.5
63093   414    2340     2.0
...     ...     ...     ...
19702   129    1073     4.0
68132   438    8869

In [4]:
# Point 1: report the number of (user,item,rating) items in each file.

def compteur(f) : 
    compt_users = -1
    compt_items = -1
    compt_ratings = -1
    users = []
    items = []
    for row in f :
        compt_ratings = compt_ratings + 1
        if row.split(',')[1] not in users :
            users.append(row.split(',')[1])
            compt_users = compt_users + 1

        if row.split(',')[2] not in items :
            items.append(row.split(',')[2])
            compt_items = compt_items + 1
    return(compt_users, compt_items, compt_ratings)

for i in range(5) :
    with open('train-%d.csv' % (i,), 'r') as f :
        print('train-%d.csv' % (i,), ' : ', compteur(f))

for i in range(5) :
    with open('test-%d.csv' % (i,), 'r') as f :
        print('test-%d.csv' % (i,), ' : ', compteur(f))


train-0.csv  :  (610, 8985, 80668)
train-1.csv  :  (610, 8995, 80669)
train-2.csv  :  (610, 8971, 80669)
train-3.csv  :  (610, 8947, 80669)
train-4.csv  :  (610, 8967, 80669)
test-0.csv  :  (610, 5126, 20168)
test-1.csv  :  (610, 5100, 20167)
test-2.csv  :  (610, 5119, 20167)
test-3.csv  :  (610, 5125, 20167)
test-4.csv  :  (610, 5127, 20167)


In [5]:
def convert_DF_to_RDD(DF):
    """ 
    This function converts a rating dataframe into a dictionary of lists
    Args:
        DF: a dataframe which contains 'user', 'item' and 'rating' columns
    Returns:
        RDD: a dictionary which contains 
            'total': the total number of rating
            'users': a list of user id for each rating
            'items': a list of item id for each rating
            'ratings': a list of ratings
    """ 
    RDD = {'total':0,'users':[],'items':[],'ratings':[]} 
    
    # transformation du df en dict
    RDD = DF.to_dict('list') 
    del RDD['Unnamed: 0']
    RDD['total'] = DF.shape[0]
    
    # création de la liste de films qui ne sont pas notés
    lst_items.sort()
    film_non_note = []
    
    for k in range(max(lst_items)) :
        if k not in lst_items :
            film_non_note.append(k)
    
    # réindiçage
    for i in range(len(RDD['user'])) :
        RDD['user'][i] = RDD['user'][i] - 1
    
    for j in range(len(RDD['item'])) : 
        k = 0
        while k < len(film_non_note) and film_non_note[k] < RDD['item'][j] :
            k = k + 1
        RDD['item'][j] = RDD['item'][j] - k
    
    return RDD

In [6]:
# read the csv files from a partition of the cross-validation
# convert them to RDD using convert_DF_to_RDD

train0 = pd.read_csv('train-0.csv')
RDD = convert_DF_to_RDD(train0)
print(RDD)

{'user': [131, 353, 390, 110, 561, 447, 297, 233, 199, 159, 291, 20, 240, 483, 465, 47, 102, 79, 374, 65, 447, 327, 176, 436, 324, 355, 356, 413, 307, 447, 159, 324, 225, 273, 110, 549, 287, 473, 102, 601, 225, 476, 413, 447, 593, 351, 494, 601, 265, 482, 273, 103, 494, 176, 139, 479, 23, 194, 293, 65, 379, 447, 56, 367, 553, 483, 312, 393, 306, 413, 381, 487, 287, 424, 585, 311, 281, 81, 434, 42, 479, 516, 198, 473, 424, 576, 509, 578, 90, 287, 578, 598, 108, 179, 185, 602, 273, 351, 598, 554, 201, 598, 177, 104, 607, 6, 317, 138, 22, 599, 571, 199, 579, 103, 102, 580, 602, 185, 16, 61, 364, 526, 558, 566, 364, 413, 583, 20, 390, 571, 447, 102, 152, 609, 5, 226, 521, 550, 566, 223, 21, 559, 566, 333, 120, 584, 413, 13, 221, 437, 492, 81, 247, 468, 513, 599, 566, 183, 473, 433, 6, 473, 223, 482, 88, 0, 121, 367, 541, 469, 340, 155, 461, 367, 544, 523, 543, 605, 554, 473, 424, 166, 386, 236, 394, 262, 124, 437, 474, 560, 233, 461, 572, 165, 317, 297, 119, 199, 239, 559, 17, 131, 238, 57

# B. Gradient-descent algorithm

Based on the preprocessing, we are going to develop a method to find optimal P and Q on training data. It contains: 

1. compute the objective and the gradient of the objective function
2. implement the gradient-descent algroithm
3. measure the speed of this method

In [7]:
# assume now you have obtained trainRDD and testRDD
# Compute the objective funtion

In [43]:
def h(a) :
    for k in range(len(a)) :
        if a[k] < 0 :
            a[k] = 0
    
    return a

def computeMSE(x, RDD=RDD, la=0):
    """ 
    This function computes regularized Mean Squared Error (MSE)
    Args:
        RDD: a dict of list of userID, itemID, Rating
        P: user's features matrix (M by K)
        Q: item's features matrix (N by K)
        la: lambda parameter for regulization (see definition in the project description)
    Returns:
        mse: mean squared error
    """ 
    
    P = x[0]
    Q = x[1]
    
    ### TODO
    R = bsr_matrix((RDD['rating'], (RDD['user'], RDD['item']))).toarray()
    print(np.shape(R))
    
    #La formule est dans le doc d'explication du projet
    A = R - np.dot(P,Q.T) 
    A = A*A
    mse = 0
    for i in range(M) :
        for j in range(N) : 
            mse = mse + A[i,j] + la*(np.linalg.norm(h(P[i, :]))**2 + np.linalg.norm(h(Q[j, :]))**2)

    return mse/(M*N)

In [42]:
# use a random P and Q to test the function computeMSE
# Point 2: report MSE with LAMBDA=0

K = 5 # rank parameter
P = np.random.rand(M,K) # user's features matrix (M by K)
Q = np.random.rand(N,K) # item's features matrix (N by K)

print(type(P))
### TODO
print('mse avec lamba = 0 : ', computeMSE(RDD,P,Q,la=0))

<class 'numpy.ndarray'>


TypeError: computeMSE() got multiple values for argument 'la'

In [44]:
# Compute the gradient of the objective funtion
def computeGradMSE(x, RDD=RDD, la=0):
    """ 
    This function computes the gradient of regularized MSE with respect to P and Q
    Args:
        RDD: a dict of list of userID, itemID, Rating
        P: user's features matrix (M by K)
        Q: item's features matrix (N by K)
    Returns:
        gradP, gradQ: gradient of mse with respect to each element of P and Q 
    """
    
    P = x[0]
    Q = x[1]
    
    R = bsr_matrix((RDD['rating'], (RDD['user'], RDD['item']))).toarray()
    
    ### TODO
    gradP = 0
    gradQ = 0
    
    for i in range(M) :
        for j in range(N) : 
            gradP += - 2*(la*h(P[i]) - (R[i, j] - np.dot(P[i], Q[j].T))*Q[j])
            gradQ += - 2*(la*h(Q[j]) - (R[i, j] - np.dot(P[i], Q[j].T))*P[i])
        
        if i%100 == 0 :
            print(i, '/600')
            
    return [gradP/M/N, gradQ/M/N]
    

In [46]:
# implement the (steepest) gradient-descent algorithm

def GD(RDD, M, N, K, MAXITER=50, GAMMA=0.001, LAMBDA=0.05, adaptive=0):
    """ 
    This function implemnts the gradient-descent method to minimize the regularized MSE with respect to P and Q
    Args:
        RDD: a dict of list of users, items, ratings
        M: number of users
        N: number of items
        K: rank parameter
        MAXITER: maximal number of iterations (epoches) of GD 
        GAMMA: step size of GD
        LAMBDA: regulization parameter lambda in the mse loss
        adaptive: if 0 then use constant step size GD, 
                  if 1 then use line search to choose the step size automatically
    Returns:
        P: optimal P found by GD
        Q: optimal Q found by GD
        lreg_mse: a list of regulized mse values evaluated on RDD, after each iteration
        lmse: a list of mse values, evaluated on RDD after each iteration
        other scores for analysis purpose
    """ 
    print('initialisation')
    #initialization
    eta = 10e-14
    eps1 = 10e-6
    eps2 = 10e-12

    P0 = np.random.rand(M,K)
    Q0 = np.random.rand(N,K)

    
    flag = 0
    i = 0
    
    print('début de la ', i+1, 'ère descente')

    x0 = [P0, Q0]
    gradMSE0 = computeGradMSE(x0, RDD,LAMBDA)
    d0 = [-gradMSE0[0], -gradMSE0[1]]
    
    x = [x0]
    dx = d0
    
    lmse = [computeMSE(x0, RDD, 0)]
    lreg_mse = [computeMSE(x0, RDD, LAMBDA)]

    while flag == 0: 
        
        print('calcul step size')
        if adaptive == 1 :
            x = [x[i][0], x[i][1]]
            ss = line_search(computeMSE, computeGradMSE, x, dx)[0]
        else : 
            ss = GAMMA

        print('calcul nouveau x')
        x.append(x[i] + [ss*i for i in dx])
        
        print('verif flag')
        if np.linalg.norm(dx) <= eps1*(np.linalg.norm(d0) + eta) : #disparition du gradient
            flag = 1  
        elif np.linalg.norm([ss*i for i in dx[0]]) <= eps2*(np.linalg.norm(x[i][0]) + eta) :  #stagnation en P
            flag = 2
        elif np.linalg.norm([ss*i for i in dx[1]]) <= eps2*(np.linalg.norm(x[i][1]) + eta) :  #stagnation en Q
            flag = 2
        elif i == MAXITER - 1:  #iterations max
            flag = 3
        else : 
            i += 1
            print('début de la ', i+1, 'eme descente')
        
        print('calcul de la descente')
        gradMSEx = computeGradMSE(x[i], RDD, LAMBDA)
        dx = [-gradMSEx[0], -gradMSEx[1]]
        
        print('calcul scores')
        #scores
        lmse.append(computeMSE(x[i], RDD, 0))
        lreg_mse.append(computeMSE(x[i], RDD, LAMBDA))
            
    P = x[i][0]
    Q = x[i][1]
    
    return P, Q, lreg_mse, lmse, flag


In [31]:
# Compare GD constant step size with GD line search step size
GD_constant = GD(RDD, M, N, K, MAXITER=50, GAMMA=0.001, LAMBDA=0.05, adaptive=0)
print('GD constant step size', GD_constant)

initialisation
début de la  1 ère descente
0 /600


KeyboardInterrupt: 

In [47]:
GD_ls = GD(RDD, M, N, K, MAXITER=50, GAMMA=0.001, LAMBDA=0.05, adaptive=1)
print('GD line search step size', GD_ls)

initialisation
début de la  1 ère descente
0 /600
100 /600
200 /600
300 /600
400 /600
500 /600
600 /600
(610, 9724)
(610, 9724)
calcul step size
0 /600
100 /600
200 /600
300 /600
400 /600
500 /600
600 /600


ValueError: shapes (2,5) and (2,5) not aligned: 5 (dim 1) != 2 (dim 0)

In [ ]:
# Point 3: Make plots to show how (regularized) MSE changes with respect to GD iterations 
# Mention your initialization of P,Q, and the stopping criterion of GD

lreg_mse = GD_constant[2]
plot(lreg_mse)

# C. Performance evaluation
1. Compute RMSE score. You should use lenskit.metrics.predict.rmse for a fair comparison. Analyze both the training and test score on the 5 cross-validation partitions. 
2. Compare with a baseline method called Bias. Tune the hyper-parameters such as K and lambda to see if you can obtain a smaller RMSE. Try to explain why. 

In [ ]:
# Point 4: Report RMSE, together with your choice of K,LAMBDA and GD parameters
def evaluteRMSE(RDD,P,Q):
    """ 
    This function computes the root MSE score on the rating of RDD. It compares the rating of each (i,j)
    in RDD, with the prediction made by <p_i,q_j>. 
    Args:
        RDD: a dict of list of users, items, ratings
        P: optimal P found by GD
        Q: optimal Q found by GD
    Returns:
        RMSE: the RMSE score
    """
    R = bsr_matrix((RDD['rating'], (RDD['user'], RDD['item']))).toarray()
    
    return rmse(np.dot(P, Q.T), R)

### TODO


In [ ]:
# Compare the performance with a baseline method called Bias
# see in https://lkpy.readthedocs.io/en/stable/bias.html
# Point 5:  report the RMSE of the baseline method, and analyze the results
# Hint: use read_csv in panda to read you csv data

algo = Bias()
algo.fit(train)
preds = predict(algo, test)
user_metric(preds, metric=rmse)



### TODO
